In [15]:
import psycopg2
import sys

import pandas as pd
import numpy as np
from matplotlib.pyplot import xticks
import matplotlib.pyplot as plt
plt.rc("font",size=14)
%matplotlib inline

import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid",color_codes=True)


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve

#from scipy.stats import chi2_contingency
#from scipy.stats import chi2
#import math

import pickle
from joblib import dump
import joblib

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [16]:
conn = psycopg2.connect(
                host="dwh-production.db.eigensonne.de",
                port="5432",
                database="eigensonne_dwh",
                user="amareid",
                password="ey4cMnS6Chni29G2H2iL",
                sslmode="require")


In [17]:


query = """
    select *
       from eigensonne_dwh.public.new_data_lead_scoring """
    
cr = conn.cursor()
result = cr.execute(query)
cols = []

for col in cr.description:
    cols.append(col[0])
    
final_list=[]
for val in cr.fetchall():
    final_list.append(dict(zip(cols,val)))
    
cr.close()
base=pd.DataFrame(final_list)
#base=base.drop('has_been_wfp',axis=1)
base.head()


,id,lead_source,lead_medium,bundesland,record_type_name_c,has_opened,has_clicked_non_photo_url,time_in_queue
0,00Q5J000002rJqPUAU,Website,cpc,Hessen,Regular,0,0,0
1,00Q5J000002rxQKUAY,KP,affiliate,Mecklenburg-Vorpommern,Rent,0,0,2
2,00Q5J000002rzx2UAA,Wattfox-Premium,affiliate,Nordrhein-Westfalen,Regular,0,0,0
3,00Q5J000003a4nKUAQ,Wattfox-Premium,affiliate,Brandenburg,Regular,0,0,0
4,00Q5J000003a56DUAQ,Wattfox-Premium,affiliate,Baden-Württemberg,Regular,1,0,0


In [18]:
data = base.copy(deep=True)

In [19]:
# Data Preparation

In [20]:


dummy1 = pd.get_dummies(data[['record_type_name_c','lead_source',
                           'lead_medium','bundesland'
                             ]], drop_first=False)

dummy1.head()

,record_type_name_c_Regular,record_type_name_c_Rent,lead_source_Calendly,lead_source_Hausfrage,lead_source_KP,lead_source_Recommendation,lead_source_Wattfox,lead_source_Wattfox-Premium,lead_source_Website,lead_medium_(none),lead_medium_Others,lead_medium_affiliate,lead_medium_cpc,lead_medium_display,lead_medium_nativead,lead_medium_organic,lead_medium_paidsocial,lead_medium_referral,bundesland_Baden-Württemberg,bundesland_Bayern,bundesland_Berlin,bundesland_Brandenburg,bundesland_Hamburg,bundesland_Hessen,bundesland_Mecklenburg-Vorpommern,bundesland_Niedersachsen,bundesland_Nordrhein-Westfalen,bundesland_Rheinland-Pfalz,bundesland_Sachsen,bundesland_Sachsen-Anhalt,bundesland_Schleswig-Holstein,bundesland_Thüringen,bundesland_Unknown/Others
0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
# Adding the results to the master dataframe
data = pd.concat([data, dummy1], axis=1)
data.head()

,id,lead_source,lead_medium,bundesland,record_type_name_c,has_opened,has_clicked_non_photo_url,time_in_queue,record_type_name_c_Regular,record_type_name_c_Rent,lead_source_Calendly,lead_source_Hausfrage,lead_source_KP,lead_source_Recommendation,lead_source_Wattfox,lead_source_Wattfox-Premium,lead_source_Website,lead_medium_(none),lead_medium_Others,lead_medium_affiliate,lead_medium_cpc,lead_medium_display,lead_medium_nativead,lead_medium_organic,lead_medium_paidsocial,lead_medium_referral,bundesland_Baden-Württemberg,bundesland_Bayern,bundesland_Berlin,bundesland_Brandenburg,bundesland_Hamburg,bundesland_Hessen,bundesland_Mecklenburg-Vorpommern,bundesland_Niedersachsen,bundesland_Nordrhein-Westfalen,bundesland_Rheinland-Pfalz,bundesland_Sachsen,bundesland_Sachsen-Anhalt,bundesland_Schleswig-Holstein,bundesland_Thüringen,bundesland_Unknown/Others
0,00Q5J000002rJqPUAU,Website,cpc,Hessen,Regular,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,00Q5J000002rxQKUAY,KP,affiliate,Mecklenburg-Vorpommern,Rent,0,0,2,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,00Q5J000002rzx2UAA,Wattfox-Premium,affiliate,Nordrhein-Westfalen,Regular,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,00Q5J000003a4nKUAQ,Wattfox-Premium,affiliate,Brandenburg,Regular,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,00Q5J000003a56DUAQ,Wattfox-Premium,affiliate,Baden-Württemberg,Regular,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
data = data.drop(['lead_source', 'lead_medium',	'bundesland',
                             'record_type_name_c'], axis = 1)


In [23]:
data.head(10)

,id,has_opened,has_clicked_non_photo_url,time_in_queue,record_type_name_c_Regular,record_type_name_c_Rent,lead_source_Calendly,lead_source_Hausfrage,lead_source_KP,lead_source_Recommendation,lead_source_Wattfox,lead_source_Wattfox-Premium,lead_source_Website,lead_medium_(none),lead_medium_Others,lead_medium_affiliate,lead_medium_cpc,lead_medium_display,lead_medium_nativead,lead_medium_organic,lead_medium_paidsocial,lead_medium_referral,bundesland_Baden-Württemberg,bundesland_Bayern,bundesland_Berlin,bundesland_Brandenburg,bundesland_Hamburg,bundesland_Hessen,bundesland_Mecklenburg-Vorpommern,bundesland_Niedersachsen,bundesland_Nordrhein-Westfalen,bundesland_Rheinland-Pfalz,bundesland_Sachsen,bundesland_Sachsen-Anhalt,bundesland_Schleswig-Holstein,bundesland_Thüringen,bundesland_Unknown/Others
0,00Q5J000002rJqPUAU,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,00Q5J000002rxQKUAY,0,0,2,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,00Q5J000002rzx2UAA,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,00Q5J000003a4nKUAQ,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,00Q5J000003a56DUAQ,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,00Q5J000003ab5hUAA,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6,00Q5J000003acKSUAY,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,00Q5J000003acP8UAI,1,1,7,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,00Q5J000003aHfeUAE,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9,00Q5J000003aJThUAM,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [24]:
#train-test split 
# Putting feature variable to X
X = data.drop(['id'#,'photo_uploaded'
              ], axis=1)

scaler = StandardScaler()


X[['time_in_queue']] = scaler.fit_transform(
    X[['time_in_queue']])

X.head()

,has_opened,has_clicked_non_photo_url,time_in_queue,record_type_name_c_Regular,record_type_name_c_Rent,lead_source_Calendly,lead_source_Hausfrage,lead_source_KP,lead_source_Recommendation,lead_source_Wattfox,lead_source_Wattfox-Premium,lead_source_Website,lead_medium_(none),lead_medium_Others,lead_medium_affiliate,lead_medium_cpc,lead_medium_display,lead_medium_nativead,lead_medium_organic,lead_medium_paidsocial,lead_medium_referral,bundesland_Baden-Württemberg,bundesland_Bayern,bundesland_Berlin,bundesland_Brandenburg,bundesland_Hamburg,bundesland_Hessen,bundesland_Mecklenburg-Vorpommern,bundesland_Niedersachsen,bundesland_Nordrhein-Westfalen,bundesland_Rheinland-Pfalz,bundesland_Sachsen,bundesland_Sachsen-Anhalt,bundesland_Schleswig-Holstein,bundesland_Thüringen,bundesland_Unknown/Others
0,0,0,-0.395934,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0.203456,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,-0.395934,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,-0.395934,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,0,-0.395934,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
clf_from_joblib = joblib.load('trained_model_2.pkl') 
list_columns_model=list(clf_from_joblib.conf_int().reset_index().iloc[1:]['index'])




In [36]:
X_list=list(X.columns) 

main_list = list(np.setdiff1d(list_columns_model,X_list))
#add columns in X with 0 values
for col in main_list:
    X[col] = 0

main_list2 = list(np.setdiff1d(X_list,list_columns_model))
#remove columns form X
for col in main_list2:
    X=X.drop(col,axis=1)

# yields the elements in `list_2` that are NOT in `list_1`
X

,has_opened,has_clicked_non_photo_url,time_in_queue,record_type_name_c_Rent,lead_source_Hausfrage,lead_source_KP,lead_source_Wattfox,lead_source_Wattfox-Premium,lead_source_Website,lead_medium_Others,lead_medium_affiliate,lead_medium_cpc,lead_medium_display,lead_medium_nativead,lead_medium_organic,lead_medium_paidsocial,lead_medium_referral,bundesland_Bayern,bundesland_Berlin,bundesland_Brandenburg,bundesland_Hamburg,bundesland_Hessen,bundesland_Mecklenburg-Vorpommern,bundesland_Niedersachsen,bundesland_Nordrhein-Westfalen,bundesland_Rheinland-Pfalz,bundesland_Sachsen,bundesland_Sachsen-Anhalt,bundesland_Schleswig-Holstein,bundesland_Thüringen,bundesland_Unknown/Others,lead_source_Lition,lead_source_Others
0,0,0,-0.395934,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0.203456,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,-0.395934,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,-0.395934,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,-0.395934,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,0,0,-0.395934,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
815,0,0,-0.395934,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
816,0,0,-0.395934,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
817,0,0,-0.395934,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [37]:
X_data = sm.add_constant(X)
#X_test_sm = sm.add_constant(X_test)

In [38]:

result= clf_from_joblib.predict(X_data)
result

0      0.455161
1      0.601618
2      0.466302
3      0.508347
4      0.918100
         ...   
814    0.552496
815    0.508347
816    0.617703
817    0.466302
818    0.508347
Length: 819, dtype: float64

In [39]:
# Converting y_pred to a dataframe which is an array
score = pd.DataFrame(result)
score= score.rename(columns={ 0 : 'Lead_Score'})

score['Lead_Score'] = score.Lead_Score.map( lambda x: round(x*100))
score

,Lead_Score
0,46
1,60
2,47
3,51
4,92
...,...
814,55
815,51
816,62
817,47


In [40]:
Final= base.merge(score, how='left',left_index=True,right_index=True).reset_index(drop=True)
Final.head(500)

,id,lead_source,lead_medium,bundesland,record_type_name_c,has_opened,has_clicked_non_photo_url,time_in_queue,Lead_Score
0,00Q5J000002rJqPUAU,Website,cpc,Hessen,Regular,0,0,0,46
1,00Q5J000002rxQKUAY,KP,affiliate,Mecklenburg-Vorpommern,Rent,0,0,2,60
2,00Q5J000002rzx2UAA,Wattfox-Premium,affiliate,Nordrhein-Westfalen,Regular,0,0,0,47
3,00Q5J000003a4nKUAQ,Wattfox-Premium,affiliate,Brandenburg,Regular,0,0,0,51
4,00Q5J000003a56DUAQ,Wattfox-Premium,affiliate,Baden-Württemberg,Regular,1,0,0,92
5,00Q5J000003ab5hUAA,Hausfrage,affiliate,Rheinland-Pfalz,Regular,1,0,0,95
6,00Q5J000003acKSUAY,Website,Others,Schleswig-Holstein,Regular,1,0,0,100
7,00Q5J000003acP8UAI,Website,Others,Baden-Württemberg,Regular,1,1,7,100
8,00Q5J000003aHfeUAE,Website,(none),Niedersachsen,Regular,0,0,0,26
9,00Q5J000003aJThUAM,Wattfox,affiliate,Hessen,Rent,0,0,0,77


In [45]:
Final.loc[Final['Lead_Score']<50].shape

(173, 9)

In [46]:
Final.shape

(819, 9)